# DEA

In [15]:
import numpy as np

# np.whereの使い方
a=np.array([1,-2,3,-4,5])
np.where(a > 0)

(array([0, 2, 4], dtype=int64),)

In [16]:
from pulp import *
MEPS = 1.0e-6

def DEA_CCR(x, y, DMUs):
    # 形状を取得
    m, n = x.shape
    s, n = y.shape

    res = []
    for o in range(n):
        # 最大化問題
        prob = LpProblem('DMU_'+str(o), LpMaximize)
        
        # 変数を定義
        v = [LpVariable('v'+str(i), lowBound=0,
                        cat='Continuous') for i in range(m)]
        u = [LpVariable('u'+str(i), lowBound=0,
                        cat='Continuous') for i in range(s)]
        
        # 目的関数を定義
        prob += lpDot(u, y[:,o])
        
        #　制約条件を定義
        prob += lpDot(v, x[:,o])==1, 'Normalize'+str(o) 
        for j in range(n):
            prob += lpDot(u, y[:,j]) <= lpDot(v, x[:, j])

        prob.solve()
        
        vs = np.array([v[i].varValue for i in range(m)]) # v*
        us = np.array([u[i].varValue for i in range(s)])  # u*
        
        # 参照集合作成
        # np.whereは条件を満たすindexを返す
        print(np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<=MEPS))
        (eo,) = np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<=MEPS)
        print(eo)
        res.append([DMUs[o], value(prob.objective),
                    set(eo), tuple(vs), tuple(us)])
    return res

In [12]:
# 人口とGDP
x = np.array([[321.774, 1376.049, 64.716, 143.457,
    80.689, 64.395, 126.573, 23.969, 59.798, 35.94],
  [14682.739, 5320.232, 2676.52, 999.832, 3226.726,
    2361.317, 4780.944, 975.012, 1745.045, 1359.773]])

# 金メダル数、銀メダル数、銅メダル数
y = np.array([[46, 26, 27, 19, 17, 10, 12, 8, 8, 4],
 [37, 18, 23, 18, 10, 18, 8, 11, 12, 3],
 [38, 26, 17, 19, 15, 14, 21, 10, 8, 15]])

# 国名
DMUs = ['アメリカ', '中国', 'イギリス', 'ロシア', 'ドイツ', 'フランス',
    '日本', 'オーストラリア', 'イタリア', 'カナダ']

res = np.array(DEA_CCR(x,y, DMUs))
res

(array([2, 7], dtype=int64),)
[2 7]
(array([3], dtype=int64),)
[3]
(array([2, 7], dtype=int64),)
[2 7]
(array([3], dtype=int64),)
[3]
(array([2, 3, 7], dtype=int64),)
[2 3 7]
(array([3, 7], dtype=int64),)
[3 7]
(array([3, 7, 9], dtype=int64),)
[3 7 9]
(array([3, 7, 9], dtype=int64),)
[3 7 9]
(array([3, 7], dtype=int64),)
[3 7]
(array([3, 7, 9], dtype=int64),)
[3 7 9]


<ipython-input-12-c4c538f6423b>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(DEA_CCR(x,y, DMUs))


array([['アメリカ', 0.3697814572, {2, 7}, (0.0031077713, 0.0),
        (0.0055588983, 0.0, 0.0030018983)],
       ['中国', 0.257167573, {3}, (0.0, 0.00018796173),
        (0.0, 0.0, 0.0098910605)],
       ['イギリス', 0.9999999970000001, {2, 7}, (0.015452129, 0.0),
        (0.027639362, 0.0, 0.014925719)],
       ['ロシア', 1.0000000009999999, {3}, (0.0, 0.001000168),
        (0.0, 0.0, 0.052631579)],
       ['ドイツ', 0.567588247, {2, 3, 7}, (0.0029566373, 0.00023597662),
        (0.023233346, 0.0, 0.011508091)],
       ['フランス', 0.666693702, {3, 7}, (0.0020937321, 0.00036639474),
        (0.0, 0.037038539, 0.0)],
       ['日本', 0.4118923692, {9, 3, 7}, (0.0015484154, 0.00016817022),
        (0.0021624441, 0.0, 0.01837824)],
       ['オーストラリア', 1.000000002, {9, 3, 7}, (0.0077004196, 0.00083632678),
        (0.010754044, 0.0, 0.091396765)],
       ['イタリア', 0.581318664, {3, 7}, (0.0027384214, 0.00047921279),
        (0.0, 0.048443222, 0.0)],
       ['カナダ', 0.9999999978, {9, 3, 7}, (0.005095832, 0.00060072

In [9]:
results = [[x[0],x[1], list(res[:,0][list(x[2])])] for x in res]
# xという変数を入れたら、1列目だけを中秋t
results = sorted(results, key=lambda x: x[1], reverse=True)

In [10]:
results

[['オーストラリア', 1.000000002, ['カナダ', 'ロシア', 'オーストラリア']],
 ['ロシア', 1.0000000009999999, ['ロシア']],
 ['カナダ', 0.9999999978, ['カナダ', 'ロシア', 'オーストラリア']],
 ['イギリス', 0.9999999970000001, ['イギリス', 'オーストラリア']],
 ['フランス', 0.666693702, ['ロシア', 'オーストラリア']],
 ['イタリア', 0.581318664, ['ロシア', 'オーストラリア']],
 ['ドイツ', 0.567588247, ['イギリス', 'ロシア', 'オーストラリア']],
 ['日本', 0.4118923692, ['カナダ', 'ロシア', 'オーストラリア']],
 ['アメリカ', 0.3697814572, ['イギリス', 'オーストラリア']],
 ['中国', 0.257167573, ['ロシア']]]